# Labels preprocessing

In [ ]:
import json
import csv
import pandas as pd
import numpy as np
from scipy import sparse
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
labels_df = pd.read_csv("kunisch-labels-df.csv", sep=';', index_col=0, encoding='latin-1')
patterns = labels_df.index
labels = list(labels_df.columns)

In [ ]:
patterns_dict = {}
cols = labels_df.columns.values
mask = labels_df.gt(0.0).values
out = [cols[x].tolist() for x in mask]
assert(len(out) == len(patterns))
for i in range(0, len(out)):
  patterns_dict[patterns[i]] = out[i]

In [ ]:
labels = []

#TO-DO: replace processing by reg exp
for label in list(labels_df.columns):
  label = label.strip().lower() # remove leading and trailing whitespaces and lowerize 
  label = label.replace('"', "") # remove quotes
  label = ''.join([i for i in label if not i.isdigit()]) # remove numbers
  label = label.replace('(', "") # remove parenthesis
  label = label.replace(')', "")
  labels.append(label)

In [ ]:
labels_dict = {}
assert(len(labels) == len(labels_df.columns))
for i in range(0, len(labels)):
  labels_dict[labels_df.columns[i]] = labels[i]

In [ ]:
# Remove stop words
from gensim.parsing.preprocessing import remove_stopwords

lemmatizer = WordNetLemmatizer()

for i in range(0, len(labels)):
  label = labels[i]
  original_label = labels_df.columns[i]
  filtereds = []
  filtered = remove_stopwords(label)
  filtereds.append(filtered)
  if filtered!=label: 
    #print("Label:    {}\nFiltered: {}\n".format(label, filtered))
    if len(filtered) > 0:
      splitted = filtered.split()
      filtereds = splitted
  else:
    if label.find(" ") >= 0:
      filtereds = label.split()
    elif len(label) > 0:
      filtereds = [label]
  lemmatized = []
  for f in filtereds:
    lemma = lemmatizer.lemmatize(f)
    if f!=lemma:
      #print("Label:{}\nLemma:{}\n".format(f, lemma))
      lemmatized.append(lemma)
    else:
      lemmatized.append(f)
  labels_dict[original_label] = lemmatized

In [ ]:
for k, v in patterns_dict.items():
  new_v = []
  for old_label in v:
    new_v += labels_dict[old_label]
  patterns_dict[k] = new_v

In [ ]:
new_labels = []
for labels in patterns_dict.values():
  for label in labels:
    if label not in new_labels:
      new_labels.append(label)
print(len(new_labels))

340


In [ ]:
pre_df = {}
for pattern in patterns_dict.keys():
  pre_df[pattern] = [0] * len(new_labels)

new_df = pd.DataFrame.from_dict(pre_df).T
new_df.columns = new_labels
new_df

,pendent,teardrop,horizontal,panel,group,vertical,bar,floating,enclosing,shorter,...,light,body,bird,striped,worm,angular,raised,head,bird-seed,long
1a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1b,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1c,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1d,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1e,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96e,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96f,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96g,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96h,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for k, v in patterns_dict.items():
  for label in v:
    new_df.at[k, label] =1 

new_df

,pendent,teardrop,horizontal,panel,group,vertical,bar,floating,enclosing,shorter,...,light,body,bird,striped,worm,angular,raised,head,bird-seed,long
1a,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1b,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1c,0,0,1,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1d,0,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1e,0,0,1,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96e,0,0,1,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
96f,0,0,0,1,0,1,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
96g,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
96h,0,0,1,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [ ]:
new_df.to_csv("processed_labels.csv")